In [5]:
import io
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pickle
import pyreadr as pyreadr
import random
import re
from scipy import io as sio
from scipy import sparse
from sh import gunzip
from tqdm import tqdm
import os

In [6]:
def read_vcf(path: str) -> 'DataFrame':
    '''
    Reads vcf file contained in given path and returns as pandas df
    *** code from https://gist.github.com/dceoy/99d976a2c01e7f0ba1c813778f9db744 ***
    
    Parameters:
        path (str) -- path to a VCF file
        
    Returns:
        pandas DataFrame containing data from given VCF file
    '''
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})

def process_vcf(path: str, zipped: bool = True, get_annotations: bool = True) -> list:
    '''
    Unzips and reads VCF file, edits resulting DataFrame to contain only 
    chromosome, position, and annotation information (if specified)
    
    Parameters:
        path (str) -- path to a VCF file
        zipped (bool) -- if true, unzips VCF file [default]
        get_annotations (bool) -- if true, extracts and adds annotation data to df
        
    Returns:
        DataFrame representation of VCF file, and list of all annotation types -- if 
        get_annotations = True
    '''
    if zipped == True:
        gunzip(path)
        
    vcf = read_vcf(path)
    mutations = vcf.loc[:, "CHROM":"ALT"]
    
    if get_annotations == True:
    
        annotations = [0] * len(vcf)
        info = vcf["INFO"]
        mutation_types = []

        for i in range(len(info)):
            try:
                annotation = re.search("Variant_Classification=([A-Z*a-z*0-9*,?'?]+[\s]*[A-Z*a-z*0-9*,?'?]*)", info[i]).group(1)
                if annotation not in mutation_types:
                    mutation_types.append(annotation)
                annotations[i] = annotation
            except:
                annotations[i] = "NA"

        mutations.loc[:, "ANNOTATION"] = annotations

        return mutations, mutation_types
    
    else:
        return mutations
    
def get_mutations_by_bin(mutations: 'DataFrame', bins: 'DataFrame') -> list:
    '''
    Accepts DataFrame of mutations and DataFrame of scATAC-seq bins and creates
    list of the number of mutations per bin
    
    Parameters:
        mutations (df) -- DataFrame containing chromosome and base pair position of mutations
        bins (df) -- DataFrame containing chromosome, starting position, and ending position of scATAC-seq bins
        
    Returns:
        list of mutations per bin, length = total # of bins
    '''
    mutations_by_bin = [0]*len(bins)

    for i in tqdm(list(mutations.index.values)):
        chrom = mutations.loc[i,'CHROM']
        pos = mutations.loc[i, 'POS']
        c = f"chr{chrom}"
        chrom_bins = bins[bins["CHROM"] == c]
        try:
            bin_index = chrom_bins[(chrom_bins['start'] <= pos) & (chrom_bins['end'] >= pos)].index[0]
            mutations_by_bin[bin_index] += 1
        except:
            continue
                
    return mutations_by_bin

def get_mutation_matrix(processed_vcfs: list, bins: 'DataFrame', filter_by: str = None) -> 'DataFrame':
    '''
    Accepts list of processed VCF files and creates a mutation df, where each row
    contains the number of mutations per bin for each VCF file
    
    Parameters:
        processed_vcfs (list) -- list of processed VCF files output by process_vcf
        bins (df) -- DataFrame containing chromosome, starting position, and ending position of scATAC-seq bins
        filter_by (str or None) -- annotation type to use in filtering mutations, None is default -- 
                                   if filter_by = None, no filtering will be performed
                                   
    Returns:
        DataFrame of mutations by bins for each VCF file provided
    '''
    if filter_by == None:
        individual_mutations = []
        for v in processed_vcfs:
            mutations = get_mutations_by_bin(v, bins)
            individual_mutations.append(mutations)
        mutation_matrix = pd.DataFrame(individual_mutations)
    
    else:
        annotated_mutations = []
        for v in processed_vcfs:
            mutations = filter_mutations(v, bins, filter_by)
            annotated_mutations.append(mutations)
        mutation_matrix = pd.DataFrame(annotated_mutations)
        
    return mutation_matrix

def filter_mutations(annotated_mutations: 'DataFrame', bins: 'DataFrame', annotation: str) -> list:
    '''
    Accepts DataFrame of mutations and DataFrame of scATAC-seq bins and finds mutations per bin
    for specified annotation type
    
    Parameters:
        annotated_mutations (df) -- DataFrame containing chromosome, base pair position, and annotation type of mutations
        bins (df) -- DataFrame containing chromosome, starting position, and ending position of scATAC-seq bins
        annotation (str) -- annotation type to use in filtering mutations
    
    Returns:
        list of mutations of given annotation type per bin
    '''
    filtered_mutations = annotated_mutations[annotated_mutations["ANNOTATION"] == annotation]
    filtered_mutations_by_bin = get_mutations_by_bin(filtered_mutations, bins)
    return filtered_mutations_by_bin

def get_cell_scores(mutation_matrix: 'DataFrame | np.ndarray', atac_sparsematrix: np.ndarray, calculate_total: bool = False) -> np.ndarray:
    '''
    Accepts DataFrame of mutations and sparse matrix from scATAC-sequencing, returns array
    of cell scores (total accessible mutations per cell / total accessibility of the cell)
    calculated by
        1. multiplying mutation array by the ATAC sparse matrix
        2. dividing product of #1 by the summed ATAC sparse matrix
        
    Parameters:
        mutation_matrix (DataFrame | np.ndarray) -- matrix of mutations per cell, each rows could be 
                                                    individuals or mutation types
        atac_sparsematrix (np.ndarray) -- binary sparse matrix from scATAC data
        calculate_total (bool) -- if True, a final row of the sum of all rows in the matrix will be
                                  added to the matrix prior to multiplying and dividing (default = False)
        
    Returns:
        matrix of cell scores, as np.ndarray
    '''
    if calculate_total == True:
        totals = []
        for i in range(mutation_matrix.shape[1]):
            totals.append(sum(mutation_matrix.loc[:,i]))
        mutation_matrix = np.r_[mutation_matrix, [totals]]
    
    accessible_mutations = mutation_matrix @ atac_sparsematrix
    atac_summed = atac_sparsematrix.sum(axis=0)
    cell_scores = np.divide(accessible_mutations, atac_summed)
    return cell_scores

def get_pagerankdicts(cellscore_matrix: 'np.ndarray', atac_cellnames: 'DataFrame') -> list[dict]:
    '''
    Prepares personalization dictionaries for use in seeding the PageRank algorithm
    
    Parameters:
        cellscore_matrix (np.ndarray) -- numpy matrix of cell scores, where rows are individuals
                                         or mutation types
        ATAC_cellnames (DataFrame) -- pandas DataFrame of all scATAC-seq cell names contained in graph, 
                                      should be only column in df
                                      
    Returns:
        list of dictionaries to be used to personalize PageRank
    '''
    pagerank_dicts = []

    for i in range(len(cellscore_matrix)):

        row = cellscore_matrix[i]
        row_dict = {}

        for k in range(row.shape[1]):
            row_dict[atac_cellnames.iloc[k]] = row[0,k]

        pagerank_dicts.append(row_dict)
    
    return pagerank_dicts

def get_pagerank_csv(G: 'networkx graph obj', pagerank_dicts: list, rna_cellnames: 'DataFrame', path: str, column_names: list[str]) -> None:
    '''
    Performs personalized PageRank for all dictionaries provided using given graph, and
    saves results as a csv file for further analysis and data visualization
    
    Parameters:
        G (networkx graph object) -- networkx graph to be used to perform PageRank
        pagerank_dicts (list) -- list of dictionaries to be used to personalize PageRank
        rna_cellnames (DataFrame) -- DataFrame containing cell names from scRNA-seq cells found in given graph
        path (str) -- path to directory and filename that csv file will be saved to
        column_names (list[str]) -- list of names of columns to be used in output csv file
        
    Returns:
        None
    '''
    csv = pd.DataFrame({"cell": rna_cells})
    
    for i in range(len(pagerank_dicts)):
        pr = nx.pagerank(G, personalization = pagerank_dicts[i])
        pr = list(pr.values())
        pr = pr[0:len(rna_cellnames)]
        csv[column_names[i]] = pr
    
    csv = csv.set_index("cell")
    csv.to_csv(path)
    
def artificially_mutate(bins: 'DataFrame', chromosome: str, bp_start: int, bp_end: int, mutation_max: int, random: bool = False) -> list:
    '''
    Artificially mutates the scATAC-seq bins for a specific chromosome based on a given bp interval and max number 
    of mutations (randomized or not) 
    
    Parameters:
        bins (df) -- DataFrame containing chromosome, starting position, and ending position of scATAC-seq bins
        chromosome (str) -- number or letter corresponding to chromosome to mutate, as string
        bp_start (int) -- starting bp for artificial mutations
        bp_end (int) -- ending bp for artificial mutations
        mutation_max (int) -- maximum number of mutations to be assigned to each bin in range; if random = False, 
                              all bins will be assigned this number; if random = True, bins will be assigned a 
                              random value from 1 to this number
        random (bool) -- default = False; if True, each bin in the bp range will be assigned a random number of
                         mutations
        
    Returns:
        list of artificial mutations per bin, length = total # of bins
    '''
    mutations_by_bin = [0]*len(ArchR_bins)
    
    c = f"chr{chromosome}"
    chrom_bins = bins[bins["CHROM"] == c]

    for i in tqdm(range(len(bins))):
        start = bins.loc[i, 'start']
        end = bins.loc[i, 'end']
        if start <= bp_end and end >= bp_start:
            if random == True:
                integer = random.randint(1, mutation_max)
                mutations_by_bin[i] += integer
            else:
                mutations_by_bin[i] += 1
                
    return mutations_by_bin

In [7]:
def main():
    
    # load scATAC-seq binary sparsematrix here
    with open('atac_sparsematrix.pickle', 'rb') as f:
        atac_sparsematrix = pickle.load(f)
        
    # load scATAC-seq bins here
    with open('ArchR_bins.pickle', 'rb') as f:
        bins = pickle.load(f)
    
    # load scATAC-seq cell names here
    atac_cellnames = pd.read_csv("/projectnb/paxlab/agokhale/MPAL/mpal_atac_cells.csv")
    atac_cellnames = atac_cellnames.iloc[:,1]

    # load scRNA-seq cell names here
    rna_cellnames = pd.read_csv('/projectnb/paxlab/agokhale/MPAL/mpal_rna_cells.csv')
    rna_cellnames = rna_cellnames.iloc[:,1]
    
    # load list of VCF files
    vcf_files = [f for f in os.listdir('/projectnb/paxlab/DATA/ICGC/WGS/LAML-KR/') if re.search('.consensus.20160830.somatic.snv_mnv.vcf$', f)]
    
    # load network
    with open("mpal_network_from_ameya.pickle", "rb") as f:
        G_mpal = pickle.load(f)
    
    # reading and processing each vcf file in vcf_files -- getting annotations, files unzipped
    processed_vcfs = []
    mutationtype_lists = []
    v1 = [vcf_files[0]]
    for v in v1:
        path = f"/projectnb/paxlab/DATA/ICGC/WGS/LAML-KR/{v}"
        vcf = process_vcf(path, zipped = False, get_annotations = True)
        processed_vcfs.append(vcf[0])
        mutationtype_lists.append(vcf[1])
    print("VCF files processed and annotated")
    
    # get cumulative mutation type list
    for l in range(1, len(mutationtype_lists)):
        if mutationtype_lists[0] != mutationtype_lists[l]:
            mutationtype_lists[0] = mutationtype_lists[0] + list(set(mutationtype_lists[l]) - set(mutationtype_lists[0]))
            
    mutation_types = mutationtype_lists[0]
    print(f"all mutation types: {mutation_types}")
    
    # get mutations per individual -- all mutation types
    mutation_matrix = get_mutation_matrix(processed_vcfs, bins)
    print(f"built mutation matrix -- shape = {mutation_matrix.shape}")
    
    # calculate accessibility normalized cell scores for mutation matrix -- all mutation types
    cell_scores = get_cell_scores(mutation_matrix, atac_sparsematrix, calculate_total = True)
    print(f"got cell score matrix -- shape = {cell_scores.shape}")
    
    # create pagerank dictionaries -- all mutation types
    pagerank_dicts = get_pagerank_dicts(cell_scores, atac_cellnames)
    print(f"got {len(pagerank_dicts)} pagerank dictionaries")
    
    # get pagerank csv file -- all mutation types
    get_pagerank_csv(G_mpal, pagerank_dicts, rna_cellnames, "/projectnb/paxlab/randers1/pagerank_all.csv", ["all1", "all2", "all3", "all4", "all5", "all6", "all7", "all8", "all"])
    print("PageRank is complete, results can be found in the csv file in the directory indicated")
    
main()

VCF files processed and annotated
all mutation types: ['lincRNA', 'Missense', 'IGR', 'Intron', "3'UTR", 'Silent', "5'Flank", 'RNA', 'Nonsense', "5'UTR", 'Splice']


100%|██████████| 4228/4228 [20:23<00:00,  3.46it/s]


built mutation matrix -- shape = (1, 6072620)
got cell score matrix -- shape = (2, 44514)


NameError: name 'get_pagerank_dicts' is not defined